### Import Modules

In [1]:
# Shapefile reading and manipulation

import fiona
from shapely.geometry import shape, Point
from shapely import speedups
speedups.enable()

# Data manipulation
import pandas
import numpy

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


### Set paths

In [2]:
points = "/users/danielcorcoran/desktop/test_coordinates_10K.csv"
lga_shapefile = "/users/danielcorcoran/desktop/github_repos/python_nb_data_spatial/data/LGA_ABS_16/LGA_2016_AUST.shp"
meshblock_shapefile = "/users/danielcorcoran/desktop/github_repos/python_nb_data_spatial/data/1270055001_mb_2016_vic_shape/MB_2016_VIC.shp"

### Import data and preview

In [3]:
#data = pandas.read_csv("/users/danielcorcoran/desktop/test_coordinates_10K.csv")

In [4]:
#Create test data
numpy.random.seed(0)

row_count = 20000

data = pandas.concat([pandas.Series([numpy.random.uniform(140,150) for n in range(row_count)]), 
                      pandas.Series([numpy.random.uniform(-32,-40) for n in range(row_count)])], 
                     axis = 1)

data.columns = ['longitude', 'latitude']
data.head()

,longitude,latitude
0,145.488135,-35.137384
1,147.151894,-32.329253
2,146.027634,-39.386405
3,145.448832,-35.249880
4,144.236548,-39.554257


In [5]:
print('Checking data shape:\n{}'.format(data.shape))

Checking data shape:
(20000, 2)


In [6]:
print('Checking data headers:\n{}'.format(list(data.columns)))

Checking data headers:
['longitude', 'latitude']


In [7]:
long_name = 'longitude'
lat_name = 'latitude'

### Create list of polygons containing Victoria only from LGA shapefile

In [8]:
all_polygons = [polygon for polygon in fiona.open(lga_shapefile)]

In [9]:
vic_polygons = []

for polygon in all_polygons:
    if polygon['properties']['STE_NAME16'] == 'Victoria' and polygon['geometry'] is not None:
        vic_polygons.append(polygon)
        
print("Found {} polygons in shapefile within victoria.".format(len(vic_polygons)))

Found 80 polygons in shapefile within victoria.


### Build spatial tree

In [10]:
from rtree import index
rtree_index = index.Index()

In [11]:
simplified_polygons = []
tolerance = 0.0005

for position, polygon in enumerate(vic_polygons):
    rtree_index.insert(position, shape(polygon['geometry']).bounds)
    shapeobject = shape(polygon['geometry'])
    simplified_polygons.append(shapeobject.simplify(tolerance = tolerance))

### Process  (with apply)

In [12]:
# Create function to return the properties for a shapefile if a match is found in rtree_index
def return_properties(point):

    for index in rtree_index.intersection(point.coords[0]):
        if point.within(shape(vic_polygons[index]['geometry'])):
        #if point.within(simplified_polygons[index]):
            return vic_polygons[index]['properties']

In [13]:
# Create new features with point objects using long and lat features
data['point'] = (list(zip(data[long_name], data[lat_name])))
data["shapely_point"] = data['point'].apply(Point)

In [14]:
from multiprocessing import Pool
p = Pool(4)

In [15]:
%%time

data['results'] = list(p.map(return_properties, data["shapely_point"]))

# Join original data with results series split into a new dataframe
# (one column for each property)
data = pandas.concat([data, data['results'].apply(pandas.Series)], axis = 1)

# Drop irrelevant columns
data.drop(['shapely_point', 'results', 'point'], axis = 1, inplace = True)

CPU times: user 5.82 s, sys: 148 ms, total: 5.97 s
Wall time: 15.1 s


In [16]:
print(data.head())

    longitude   latitude LGA_CODE16 LGA_NAME16 STE_CODE16 STE_NAME16  \
0  145.488135 -35.137384        NaN        NaN        NaN        NaN   
1  147.151894 -32.329253        NaN        NaN        NaN        NaN   
2  146.027634 -39.386405        NaN        NaN        NaN        NaN   
3  145.448832 -35.249880        NaN        NaN        NaN        NaN   
4  144.236548 -39.554257        NaN        NaN        NaN        NaN   

   AREASQKM16  
0         NaN  
1         NaN  
2         NaN  
3         NaN  
4         NaN  


In [17]:
# Use 'MB_CODE16' for mesh blocks and 'LGA_CODE16' for local government areas
column_header = 'LGA_CODE16'

print('Points found {}. Total Points {}. % Points found {:.2%}'.format(data[column_header].notnull().sum(),
                                                                        data[column_header].shape[0],
                                (data[column_header].notnull().sum()/data[column_header].shape[0])))

Points found 5678. Total Points 20000. % Points found 28.39%


### Export

In [18]:
data.to_csv('/users/danielcorcoran/desktop/spatial_join_results.csv', index_label='row_index')